In [73]:
import numpy
import pandas as pd

pd.set_option("display.precision",5)
df = pd.read_csv('downloader/testidata_u_07102018.csv',sep='#')
df.dtypes

#df.describe()

Havainto id            int64
Laji                  object
Pvm1                  object
Pvm2                  object
Kello_hav_1           object
Kello_hav_2           object
Kunta                 object
Paikka                object
X-koord              float64
Y-koord              float64
Tarkkuus              object
X-koord-linnun       float64
Y-koord-linnun       float64
Tarkkuus_linnun       object
Paikannettu           object
Lisätietoja           object
Atlaskoodi           float64
Tallentaja            object
Tallennusaika         object
Havainnoijat          object
Salattu               object
Tallenteita           object
Koontihavainto        object
Kuuluu havaintoon    float64
Määrä                float64
Kello_lintu_1         object
Kello_lintu_2         object
Sukupuoli             object
Puku                  object
Ikä                   object
Tila                  object
Lisätietoja_2         object
Parvi                float64
Bongattu              object
Pesintä       

In [70]:
def timeframe(data, days=7):
    
    dataframe = data    
    return dataframe

df_7 = timeframe(df,7)

In [59]:
sum_individuals = df['Määrä'].sum()
sum_species = df.groupby(['Laji'], as_index=False, sort=False)[["Määrä"]].sum().sort_values(by=['Määrä'],ascending=False);
print (sum_species)

                                     Laji   Määrä
19                                  tilhi  9282.0
26                           laulujoutsen  8399.0
113                            isokoskelo  6720.0
23                                  kurki  4771.0
55                              urpiainen  4151.0
0                           räkättirastas  4118.0
22                            järripeippo  3607.0
145                   rastaslaji (Turdus)  3336.0
52                                 peippo  2154.0
136                             vesilintu  2085.0
103                            tukkasotka  1878.0
44                              sinisorsa  1693.0
96                                haapana  1603.0
95                                 telkkä  1463.0
146                            pikkulintu  1356.0
1                         punakylkirastas  1151.0
102                                  tavi  1008.0
57                               varpunen   847.0
53                            talitiainen   803.0


In [60]:
print (f'havaintoja yhteensä {sum_individuals:.0f}')
print ('tallentajia',df['Tallentaja'].nunique())

havaintoja yhteensä 73885
tallentajia 189


In [61]:
top_recorders = df_7.groupby(['Tallentaja'], as_index=True, sort=False).agg({'Havainto id': 'count'}).sort_values(by=['Havainto id'],ascending=False);

print (f'{top_recorders}')

                         Havainto id
Tallentaja                          
Timo J. Leppänen                 616
Jouni Parviainen                 354
Pirkka Aalto                     268
Jukka Simula                     241
Simon Wright                     226
Jorma V.A. Halonen               212
Pekka Peltoniemi                 174
Roope Ruokonen                   166
Olli Osmonen                     153
Jorma Salo                       150
Jarno Luiro                      147
Juhani Honkola                   147
Jörgen Palmgren                  146
Jouni Aikio                      144
Eila Ylilokka                    135
Matti Välimäki                   111
Jukka Jokimäki                   110
Markku Sirkka                    107
Anssi Mäkinen                    107
Jari Laitasalo                    79
Leena Muotka                      78
Vesa Nivala                       69
Minna Kaan                        68
Juha Takalo                       65
Christer Casagrande               62
T